In [29]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

# Load the model
# model = load_model("15_point_plastic_classifier_model3.keras")
model = load_model("15_point_plastic_classifier_modelVPP_PET.keras")
# model = load_model("3_point_plastic_classifier_model.keras")



In [30]:
# # Sample data based on your provided example (modify this as needed)
# sample_data = [
#     [2493065, 4558145, 7638499, 8387631, 8388397, 8389475, 4054559, 2106185, 3340801, 5935779, 8389399, 8388961, 8386511, 6563979, 3097997]
# ]

# # Create a function to generate a representative dataset
# def representative_dataset():
#     for data_point in sample_data:
#         # Reshape and convert to float32 (modify this if your model's input format is different)
#         input_data = tf.constant(data_point, shape=(1, 15), dtype=tf.float32)
#         yield [input_data]

# # Convert the model to TFLite format with quantization
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8
# converter.representative_dataset = representative_dataset  # Specify the representative dataset

# tflite_model = converter.convert()

# # Save the TensorFlow Lite model
# with open('your_model.tflite', 'wb') as f:
#     f.write(tflite_model)

In [31]:
# data = pd.read_csv("AllTogetherPE.csv")
# data = pd.read_csv("resampled_dataset3.csv")
data = pd.read_csv("resampled_datasetVPP_PET.csv")

# Separate features and labels
X = data.drop(columns=["Material_Type"])
y = data["Material_Type"]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets (Normal)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=54)


# Normalize the manual input if needed
X_test_normalized = ((X_test - 0) / (8410000 - 0))
# X_test_normalized = (X_test - X_train.min()) / (X_train.max() - X_train.min())

# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)

# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -5:]
top_labels = top_predictions[:, ::-1]

# print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(len(predicted_labels)):
    num_predictions = min(5, len(np.unique(top_labels[i])))
    if y_test[i] == predicted_labels[i]:
        CorF = "Correct,"
    else:
        CorF = "False,"
    print(f"Sample {i+1}: {CorF} it was {label_encoder.classes_[y_test[i]]}")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][1-j]:.2f}% likelihood")



50/50 [==============================] - 0s 751us/step


ValueError: y contains previously unseen labels: [4]

In [ ]:

exclude_alu_indices = y != 'ALU'
X_filtered = X[exclude_alu_indices]
y_filtered = y_encoded[exclude_alu_indices]
# Split the filtered data into train and test sets (Filtered ALU)
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=50)
#features
X_test_normalized = ((X_test - 0) / (8410000 - 0))

# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)

# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -4:]
top_labels = top_predictions[:, ::-1]

# print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(len(predicted_labels)):
    num_predictions = min(4, len(np.unique(top_labels[i])))
    if y_test[i] == predicted_labels[i]:
        CorF = "Correct,"
    else:
        CorF = "False,"
    print(f"Sample {i+1}: {CorF} it was {label_encoder.classes_[y_test[i]]}")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][3-j]:.2f}% likelihood")


50/50 [==============================] - 0s 605us/step
Test Accuracy: 0.9906250238418579
Sample 1: Correct, it was PE
  Prediction 1: PE with 100.00% likelihood
  Prediction 2: PP with 0.00% likelihood
  Prediction 3: Other with 0.00% likelihood
  Prediction 4: Alu with 0.00% likelihood
Sample 2: Correct, it was PE
  Prediction 1: PE with 99.73% likelihood
  Prediction 2: PP with 0.27% likelihood
  Prediction 3: Other with 0.00% likelihood
  Prediction 4: Alu with 0.00% likelihood
Sample 3: Correct, it was Other
  Prediction 1: Other with 100.00% likelihood
  Prediction 2: PE with 0.00% likelihood
  Prediction 3: PP with 0.00% likelihood
  Prediction 4: Alu with 0.00% likelihood
Sample 4: Correct, it was PP
  Prediction 1: PP with 99.82% likelihood
  Prediction 2: PE with 0.18% likelihood
  Prediction 3: Other with 0.00% likelihood
  Prediction 4: Alu with 0.00% likelihood
Sample 5: Correct, it was Other
  Prediction 1: Other with 100.00% likelihood
  Prediction 2: PP with 0.00% likeli

In [ ]:
# Get manual input from the user
manual_input = np.array([[1271389,2107581,3534859,5988057,6233631,3880377,1498541,985865,1561809,2707983,4719147,6111335,5063453,2701889,1262325]])  # Replace with your input values

# Normalize the manual input if needed
normalized_manual_input = ((manual_input - 0) / (8410000 - 0))

# Make predictions on manual input
predictions = model.predict(normalized_manual_input)
print(predictions)

# Get the top prediction
top_prediction = np.argmax(predictions)
print(top_prediction)

top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
print(top_plastic_type)
likelihood = predictions[0][top_prediction] * 100

# Display the prediction for manual input
print(f"Manual Input Prediction:")
print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

1/1 [==============================] - 0s 73ms/step
[[7.3998336e-30 5.0405905e-23 5.3184549e-03 9.9468154e-01]]
3
PP
Manual Input Prediction:
  Prediction: PP with 99.47% likelihood


In [33]:
import numpy as np  # Add this line to import the NumPy library
import serial
from sklearn.preprocessing import LabelEncoder


# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Initialize a variable to store the received data
received_data = ""

print(label_encoder.classes_)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()
        
        # Append the received line to the variable
        received_data += line

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in received_data.split(',')]

        # Convert the list of values into a NumPy array
        manual_input = np.array([data_values])

        # Normalize the manual input if needed
        normalized_manual_input = ((manual_input - 0) / (8410000 - 0))

        # Make predictions on manual input
        predictions = model.predict(normalized_manual_input)
        # print(predictions)

        # Get the top prediction
        top_prediction = np.argmax(predictions)
        print(top_prediction)

        top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
        print(top_plastic_type)
        likelihood = predictions[0][top_prediction] * 100

        # Display the prediction for manual input
        print(f"Manual Input Prediction:")
        print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

        received_data = ""

        # Send the top plastic type back over serial
        ser.write(top_plastic_type.encode('utf-8'))

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the serial port when done
    ser.close()

# Now, you can use the 'received_data' variable for further processing
print("total data:\n", received_data)


['Alu' 'Other' 'PE' 'PP']


SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))

In [ ]:
import numpy as np
import serial
import csv  # Import the csv module
from sklearn.preprocessing import LabelEncoder

# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Create and open a CSV file for writing
csv_file = open('PET.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in line.split(',')]

        print(data_values)

        # Write the data values to the CSV file as a new row
        csv_writer.writerow(data_values)

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the CSV file and the serial port when done
    csv_file.close()
    ser.close()

# The received data has been saved in the 'received_data.csv' file
print("Data saved to 'PP2.csv'")



[352935.0, 899897.0, 1579555.0, 3927923.0, 2657159.0, 2159383.0, 591275.0, 466279.0, 599867.0, 1220707.0, 2737465.0, 3268013.0, 2392045.0, 1378637.0, 533619.0]
[178049.0, 411443.0, 464109.0, 1666879.0, 2309841.0, 1551529.0, 527493.0, 268397.0, 273927.0, 421085.0, 1078697.0, 2038083.0, 1982619.0, 1078525.0, 401785.0]
[158793.0, 298485.0, 791337.0, 1657317.0, 844077.0, 405877.0, 264219.0, 114519.0, 226995.0, 525565.0, 1218411.0, 1271997.0, 641041.0, 339259.0, 192631.0]
[149781.0, 326853.0, 510411.0, 886343.0, 1369255.0, 592421.0, 443111.0, 226415.0, 226513.0, 408033.0, 694409.0, 1133929.0, 987415.0, 523455.0, 336871.0]
[302403.0, 450685.0, 1092925.0, 2488113.0, 2033155.0, 1296315.0, 540861.0, 295783.0, 363669.0, 763109.0, 1835087.0, 2304555.0, 1668435.0, 919719.0, 419789.0]
[172669.0, 441269.0, 829187.0, 2300511.0, 1467037.0, 1372099.0, 554457.0, 324263.0, 315719.0, 595487.0, 1239845.0, 1510369.0, 1288863.0, 905929.0, 427717.0]
[345937.0, 559469.0, 1279189.0, 2355575.0, 1906433.0, 139717

SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))